# Introduction

This notebook addresses various business problems relating to a dataset containing retail information. The full list of tasks can be found below:
- Calculate the number of invoices
- Find the number of placed and cancelled orders by month
- Calculate the monthly total sales
- Calculate the monthly sales growth as a percentage
- Find the number of active users for each month
- Find the number of new and existing users for each month
- Perform RFM segmentation

## Dataframe Setup

In [0]:
# File location and type
file_location = "/FileStore/tables/online_retail_II.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# Import sql functions
from pyspark.sql.functions import *

# Import sql window
from pyspark.sql.window import Window

# The applied options are for CSV files. For other file types, these will be ignored.
df = (spark
      .read.format(file_type)
      .option("inferSchema", infer_schema)
      .option("header", first_row_is_header)
      .option("sep", delimiter)
      .load(file_location)
      .na.drop()
      .withColumnRenamed("Customer ID","CustomerID")
     )

display(df)

Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country
489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01T07:45:00.000+0000,6.95,13085.0,United Kingdom
489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01T07:45:00.000+0000,6.75,13085.0,United Kingdom
489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01T07:45:00.000+0000,6.75,13085.0,United Kingdom
489434,22041,"""RECORD FRAME 7"""" SINGLE SIZE """,48,2009-12-01T07:45:00.000+0000,2.1,13085.0,United Kingdom
489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01T07:45:00.000+0000,1.25,13085.0,United Kingdom
489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01T07:45:00.000+0000,1.65,13085.0,United Kingdom
489434,21871,SAVE THE PLANET MUG,24,2009-12-01T07:45:00.000+0000,1.25,13085.0,United Kingdom
489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01T07:45:00.000+0000,5.95,13085.0,United Kingdom
489435,22350,CAT BOWL,12,2009-12-01T07:46:00.000+0000,2.55,13085.0,United Kingdom
489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01T07:46:00.000+0000,3.75,13085.0,United Kingdom


In [0]:
# View schema
df.printSchema()

root
-- Invoice: string (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: integer (nullable = true)
-- InvoiceDate: timestamp (nullable = true)
-- Price: double (nullable = true)
-- CustomerID: double (nullable = true)
-- Country: string (nullable = true)

In [0]:
# View summary statistics
df.describe().show()

+-------+------------------+------------------+--------------------+------------------+------------------+------------------+-----------+
summary| Invoice| StockCode| Description| Quantity| Price| CustomerID| Country|
+-------+------------------+------------------+--------------------+------------------+------------------+------------------+-----------+
 count| 824364| 824364| 824364| 824364| 824364| 824364| 824364|
 mean| 537410.8855639135|28826.655026367764| null|12.414574144431343|3.6767995788248706| 15324.63850435002| null|
 stddev|26666.396588957836| 18528.79298168221| null|188.97609900975917| 70.24138768949338|1697.4644503793172| null|
 min| 489434| 10002| DOORMAT UNION J...| -80995| 0.0| 12346.0| Australia|
 max| C581569| TEST002|ZINC WIRE SWEETHE...| 80995| 38970.0| 18287.0|West Indies|
+-------+------------------+------------------+--------------------+------------------+------------------+------------------+-----------+

## Total Invoice Amount Distribution
Task: Calculate the number of invoices

In [0]:
# Filter out cancelled records
filled = df.filter(df["Quantity"] > 0).filter(df["Price"] > 0)

# Add "Amount" column
invoices = filled.withColumn("Amount", filled["Quantity"] * filled["Price"]).groupby(["Invoice"]).sum()

# Fix column names and drop "CustomerID"
invoices = (invoices
            .drop("sum(CustomerID)")
            .withColumnRenamed("sum(Quantity)", "Quantity")
            .withColumnRenamed("sum(Price)", "Price")
            .withColumnRenamed("sum(Amount)", "Amount")
           )

# Display the resulting dataframe
display(invoices)

Invoice,Quantity,Price,Amount
489677,48,8.0,192.0
491045,64,59.5,303.2
491658,61,111.05000000000001,155.05999999999997
493542,71,59.84,118.75
493977,68,95.65,275.95
494244,4980,30.699999999999996,6711.0
494277,804,10.69,1335.92
495185,776,202.83999999999995,2507.06
495783,1,48.96,48.96
496171,14,39.849999999999994,199.29999999999998


In [0]:
# Calculate invoice count
invoice_count = invoices.count()

# Calculate average invoice amount
avg_amount = str(invoices.select(format_number(avg("Amount"),2)).first()[0])

# Print number of invoices and average invoice amount
print("There are " + str(invoice_count) + " invoices in the dataset, with an average amount of $" + avg_amount + ".")

There are 36969 invoices in the dataset, with an average amount of $479.95.

## Monthly Placed and Cancelled Orders
Task: Find the number of placed and cancelled orders by month

In [0]:
# Select Invoices and date (formatted as YYYYMM), then drop NA values
placed_cancelled_df = df.select("Invoice", date_format("InvoiceDate", "YYYYMM").alias("YYYYMM"))

# Display the resulting dataframe
display(placed_cancelled_df)

Invoice,YYYYMM
489434,200912
489434,200912
489434,200912
489434,200912
489434,200912
489434,200912
489434,200912
489434,200912
489435,200912
489435,200912


In [0]:
# Create a dataframe containing all invoices grouped by month
total_df = (placed_cancelled_df
            .withColumnRenamed("Invoice", "Total")
            .groupBy("YYYYMM").agg(expr("COUNT(DISTINCT Total)").alias("Total"))
            .orderBy(col("YYYYMM").asc())
           )

# Display the resulting dataframe
display(total_df)

YYYYMM,Total
200912,1900
201001,1296
201002,1335
201003,1907
201004,1615
201005,1768
201006,1833
201007,1713
201008,1547
201009,2041


In [0]:
# Create a dataframe containing all cancelled invoices, then group by month
cancelled_df = (placed_cancelled_df
                .filter(col("Invoice").startswith("C"))
                .withColumnRenamed("Invoice", "Cancelled")
                .groupBy("YYYYMM").agg(expr("COUNT(DISTINCT Cancelled)").alias("Cancelled"))
                .orderBy(col("YYYYMM").asc())
               )

# Display the resulting dataframe
display(cancelled_df)

YYYYMM,Cancelled
200912,388
201001,285
201002,229
201003,383
201004,286
201005,391
201006,336
201007,332
201008,254
201009,352


In [0]:
# Join total and cancelled dataframes on month
all_orders_df = (total_df
                 .join(cancelled_df,"YYYYMM","inner")
                 .orderBy(col("YYYYMM").asc())
                )

# Calculate number of placed invoices
all_orders_df = all_orders_df.withColumn("Placed", all_orders_df["Total"] - (all_orders_df["Cancelled"] * 2))

# Display the resulting dataframe
display(all_orders_df)

YYYYMM,Total,Cancelled,Placed
200912,1900,388,1124
201001,1296,285,726
201002,1335,229,877
201003,1907,383,1141
201004,1615,286,1043
201005,1768,391,986
201006,1833,336,1161
201007,1713,332,1049
201008,1547,254,1039
201009,2041,352,1337


## Monthly Sales
Task: Calculate the monthly total sales

In [0]:
# Display dataframe
display(df)

Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country
489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01T07:45:00.000+0000,6.95,13085.0,United Kingdom
489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01T07:45:00.000+0000,6.75,13085.0,United Kingdom
489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01T07:45:00.000+0000,6.75,13085.0,United Kingdom
489434,22041,"""RECORD FRAME 7"""" SINGLE SIZE """,48,2009-12-01T07:45:00.000+0000,2.1,13085.0,United Kingdom
489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01T07:45:00.000+0000,1.25,13085.0,United Kingdom
489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01T07:45:00.000+0000,1.65,13085.0,United Kingdom
489434,21871,SAVE THE PLANET MUG,24,2009-12-01T07:45:00.000+0000,1.25,13085.0,United Kingdom
489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01T07:45:00.000+0000,5.95,13085.0,United Kingdom
489435,22350,CAT BOWL,12,2009-12-01T07:46:00.000+0000,2.55,13085.0,United Kingdom
489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01T07:46:00.000+0000,3.75,13085.0,United Kingdom


In [0]:
# Create a dataframe containing month, quantity, price, and amount, grouped by month
sales_df = (df
            .select("Invoice", date_format("InvoiceDate", "YYYYMM").alias("YYYYMM"), "Quantity", "Price")
            .withColumn("Amount", df["Quantity"] * df["Price"])
            .groupBy("YYYYMM").sum()
            .withColumnRenamed("sum(Quantity)","Quantity")
            .withColumnRenamed("sum(Price)","Price")
            .withColumnRenamed("sum(Amount)","Amount")
            .orderBy(col("YYYYMM").asc())
           )

# Display the resulting dataframe
display(sales_df)

YYYYMM,Quantity,Price,Amount
200912,390286,106525.29999998609,663272.0499999978
201001,367141,105352.01199999148,531952.9020000178
201002,366317,81499.0959999923,489399.58600001415
201003,499030,182832.18100002385,635996.4810000026
201004,345590,114226.56199999303,560635.0220000078
201005,369726,99087.9099999871,559924.5500000105
201006,376237,145131.41000000422,571459.9100000149
201007,319573,105398.76999999318,562785.900000009
201008,446926,90059.35999998971,587256.4599999989
201009,474112,142728.65099999652,781033.3009999989


## Monthly Sales Growth
Task: Calculate the monthly sales growth as a percentage

In [0]:
# Add "LastMonthAmount" column
sales_growth_df = (sales_df
                   .withColumn("LastMonthAmount", lag("Amount").over(Window().partitionBy().orderBy(col("YYYYMM"))))
                   .withColumn("LastMonthAmount", coalesce("LastMonthAmount", "Amount"))
                  )

# Add "Growth" column
sales_growth_df = sales_growth_df.withColumn("Growth", sales_growth_df["Amount"] / sales_growth_df["LastMonthAmount"] - 1)

# Display the resulting dataframe
display(sales_growth_df)

YYYYMM,Quantity,Price,Amount,LastMonthAmount,Growth
200912,390286,106525.29999998609,663272.0499999978,663272.0499999978,0.0
201001,367141,105352.01199999148,531952.9020000178,663272.0499999978,-0.19798685622284318
201002,366317,81499.0959999923,489399.58600001415,531952.9020000178,-0.07999451801092394
201003,499030,182832.18100002385,635996.4810000026,489399.58600001415,0.29954437885442786
201004,345590,114226.56199999303,560635.0220000078,635996.4810000026,-0.11849351568974242
201005,369726,99087.9099999871,559924.5500000105,560635.0220000078,-0.0012672629645268385
201006,376237,145131.41000000422,571459.9100000149,559924.5500000105,0.02060163284500427
201007,319573,105398.76999999318,562785.900000009,571459.9100000149,-0.015178685062974484
201008,446926,90059.35999998971,587256.4599999989,562785.900000009,0.04348111777496477
201009,474112,142728.65099999652,781033.3009999989,587256.4599999989,0.3299697052289563


## Monthly Active Users
Task: Find the number of active users for each month

In [0]:
# Create a dataframe containing month and customer ID, grouped by month
monthly_users_df = (df
                    .select("CustomerID", date_format("InvoiceDate", "YYYYMM").alias("YYYYMM"))
                    .groupBy("YYYYMM").agg(expr("COUNT(DISTINCT CustomerID)").alias("UniqueCustomers"))
                    .orderBy("YYYYMM")
                   )

# Display the resulting dataframe
display(monthly_users_df)

YYYYMM,UniqueCustomers
200912,1045
201001,786
201002,807
201003,1111
201004,998
201005,1062
201006,1095
201007,988
201008,964
201009,1202


## New and Existing Users
Task: Find the number of new and existing users for each month

In [0]:
# Create a dataframe containing month and customer ID
new_old_users_df = df.select("CustomerID", date_format("InvoiceDate", "YYYYMM").alias("YYYYMM"))

# Display the resulting dataframe
display(new_old_users_df)

CustomerID,YYYYMM
13085.0,200912
13085.0,200912
13085.0,200912
13085.0,200912
13085.0,200912
13085.0,200912
13085.0,200912
13085.0,200912
13085.0,200912
13085.0,200912


In [0]:
# Create a dataframe containing date of first purchase for each customer
first_purchases_df = (new_old_users_df
                      .groupBy("CustomerID").agg(min("YYYYMM"))
                      .withColumnRenamed("min(YYYYMM)","FirstPurchase")
                      .orderBy("CustomerID")
                     )

# Display the resulting dataframe
display(first_purchases_df)

CustomerID,FirstPurchase
12346.0,200912
12347.0,201010
12348.0,201009
12349.0,200912
12350.0,201102
12351.0,201011
12352.0,201011
12353.0,201010
12354.0,201104
12355.0,201005


In [0]:
# Join new_old_users and first_purchases dataframes
new_old_users_df = new_old_users_df.join(first_purchases_df, "CustomerID", "inner")

# Display the resulting dataframe
display(new_old_users_df)

CustomerID,YYYYMM,FirstPurchase
12467.0,201010,201010
12467.0,201010,201010
12467.0,201010,201010
12467.0,201010,201010
12467.0,201010,201010
12467.0,201010,201010
12467.0,201010,201010
12467.0,201010,201010
12467.0,201010,201010
12467.0,201011,201010


In [0]:
# Create a dataframe containing the number of new users per month
new_users_df = (new_old_users_df
                .filter(col("FirstPurchase") == col("YYYYMM"))
                .groupBy("YYYYMM").agg(expr("COUNT(DISTINCT CustomerID)").alias("NewCustomers"))
                .orderBy("YYYYMM")
               )

# Display the resulting dataframe
display(new_users_df)

YYYYMM,NewCustomers
200912,1045
201001,394
201002,363
201003,436
201004,291
201005,254
201006,269
201007,183
201008,158
201009,242


In [0]:
# Create a dataframe containing the number of old users per month
old_users_df = (new_old_users_df
                .filter(col("FirstPurchase") < col("YYYYMM"))
                .groupBy("YYYYMM").agg(expr("COUNT(DISTINCT CustomerID)").alias("OldCustomers"))
                .union(spark.createDataFrame([("200912", 0)], ["YYYYMM", "OldCustomers"]))
                .orderBy("YYYYMM")
               )

# Display the resulting dataframe
display(old_users_df)

YYYYMM,OldCustomers
200912,0
201001,392
201002,444
201003,675
201004,707
201005,808
201006,826
201007,805
201008,806
201009,960


In [0]:
# Join new_users_df and old_users_df dataframes
new_users_df = new_users_df.withColumnRenamed("YYYYMM","YYYYMM")
old_users_df = old_users_df.withColumnRenamed("YYYYMM","YYYYMM")
combined_users_df = (new_users_df
                     .join(old_users_df, "YYYYMM", "inner")
                     .orderBy("YYYYMM")
                    )

# Display the resulting dataframe
display(combined_users_df)

YYYYMM,NewCustomers,OldCustomers
200912,1045,0
201001,394,392
201002,363,444
201003,436,675
201004,291,707
201005,254,808
201006,269,826
201007,183,805
201008,158,806
201009,242,960


## RFM Segmentation
Task: Perform RFM segmentation

In [0]:
# Create a dataframe containing invoice, month, customer ID, quantity, and price
users_df = (df
            .select("Invoice", "InvoiceDate", date_format("InvoiceDate", "YYYYMM").alias("YYYYMM"), "CustomerID", "Quantity", "Price")
            .filter(df["Quantity"] > 0)
            .filter(df["Price"] > 0)
           )

# Display the resulting dataframe
display(users_df)

Invoice,InvoiceDate,YYYYMM,CustomerID,Quantity,Price
489434,2009-12-01T07:45:00.000+0000,200912,13085.0,12,6.95
489434,2009-12-01T07:45:00.000+0000,200912,13085.0,12,6.75
489434,2009-12-01T07:45:00.000+0000,200912,13085.0,12,6.75
489434,2009-12-01T07:45:00.000+0000,200912,13085.0,48,2.1
489434,2009-12-01T07:45:00.000+0000,200912,13085.0,24,1.25
489434,2009-12-01T07:45:00.000+0000,200912,13085.0,24,1.65
489434,2009-12-01T07:45:00.000+0000,200912,13085.0,24,1.25
489434,2009-12-01T07:45:00.000+0000,200912,13085.0,10,5.95
489435,2009-12-01T07:46:00.000+0000,200912,13085.0,12,2.55
489435,2009-12-01T07:46:00.000+0000,200912,13085.0,12,3.75


In [0]:
# Create a dataframe containing the most recent purchase date for each customer
last_purchases_df = (users_df
                     .select("CustomerID", "YYYYMM")
                     .groupBy("CustomerID").agg(max("YYYYMM").alias("LastPurchase"))
                    )

# Display the resulting dataframe
display(last_purchases_df)

CustomerID,LastPurchase
12467.0,201010
12493.0,201106
12671.0,201004
12737.0,201007
13094.0,201111
13533.0,201106
13607.0,201110
13918.0,201110
13956.0,201112
13973.0,201102


In [0]:
# Update users_df dataframe by joining with last_purchases_df
users_df = (users_df
            .join(last_purchases_df, "CustomerID", "inner")
            .filter(users_df["Quantity"] > 0)
            .filter(users_df["Price"] > 0)
            .withColumn("Amount", users_df["Quantity"] * users_df["Price"])
            .drop("Quantity", "Price")
            .orderBy("CustomerID")
           )

# Display the resulting dataframe
display(users_df)

CustomerID,Invoice,InvoiceDate,YYYYMM,LastPurchase,Amount
12346.0,494450,2010-01-14T13:50:00.000+0000,201001,201101,22.5
12346.0,541431,2011-01-18T10:01:00.000+0000,201101,201101,77183.6
12346.0,513774,2010-06-28T13:53:00.000+0000,201006,201101,7.49
12346.0,499763,2010-03-02T13:08:00.000+0000,201003,201101,3.25
12346.0,491744,2009-12-14T11:02:00.000+0000,200912,201101,22.5
12346.0,513774,2010-06-28T13:53:00.000+0000,201006,201101,7.49
12346.0,513774,2010-06-28T13:53:00.000+0000,201006,201101,7.49
12346.0,492718,2009-12-18T10:47:00.000+0000,200912,201101,22.5
12346.0,513774,2010-06-28T13:53:00.000+0000,201006,201101,7.49
12346.0,513774,2010-06-28T13:53:00.000+0000,201006,201101,7.49


In [0]:
# Create dataframe containing the monetary value for each customer
monetary_df = (users_df
               .select("CustomerID", "Amount")
               .groupBy("CustomerID").agg(sum("Amount").alias("Monetary"))
               .orderBy("CustomerID")
              )

# Display the resulting dataframe
display(monetary_df)

CustomerID,Monetary
12346.0,77556.46
12347.0,5633.3199999999915
12348.0,2019.4
12349.0,4428.690000000002
12350.0,334.40000000000003
12351.0,300.93
12352.0,2849.8400000000015
12353.0,406.75999999999993
12354.0,1079.4
12355.0,947.61


In [0]:
# Create dataframe containing the frequency value for each customer
frequency_df = (users_df
                .select("CustomerID", "Invoice")
                .groupBy("CustomerID").agg(expr("COUNT(DISTINCT Invoice)").alias("Frequency"))
                .orderBy("CustomerID")
               )

# Display the resulting dataframe
display(frequency_df)

CustomerID,Frequency
12346.0,12
12347.0,8
12348.0,5
12349.0,4
12350.0,1
12351.0,1
12352.0,10
12353.0,2
12354.0,1
12355.0,2


In [0]:
# Create dataframe containing the recency value for each customer
recency_df = (users_df
              .select("CustomerID", "InvoiceDate")
              .withColumn("Recency", datediff(current_date(), users_df["InvoiceDate"]))
              .groupBy("CustomerID").agg(min("Recency").alias("Recency"))
              .orderBy("CustomerID")
             )

# Display the resulting dataframe
display(recency_df)

CustomerID,Recency
12346.0,4074
12347.0,3751
12348.0,3824
12349.0,3767
12350.0,4059
12351.0,4124
12352.0,3785
12353.0,3953
12354.0,3981
12355.0,3963


In [0]:
# Join the recency_df, frequency_df, and monetary_df dataframes
rfm_overview_df = (recency_df
                   .join(frequency_df, "CustomerID", "inner")
                   .join(monetary_df, "CustomerID", "inner")
                   .orderBy("CustomerID")
                  )

# Display the resulting dataframe
display(rfm_overview_df)

CustomerID,Recency,Frequency,Monetary
12346.0,4074,12,77556.46
12347.0,3751,8,5633.3199999999915
12348.0,3824,5,2019.4
12349.0,3767,4,4428.690000000002
12350.0,4059,1,334.40000000000003
12351.0,4124,1,300.93
12352.0,3785,10,2849.8400000000015
12353.0,3953,2,406.75999999999993
12354.0,3981,1,1079.4
12355.0,3963,2,947.61


In [0]:
# Find frequency scores
rfm_table = (rfm_overview_df
             .select("CustomerID", "Recency", "Frequency", "Monetary", ntile(5).over(Window.partitionBy().orderBy(rfm_overview_df.Recency)).alias("RecencyScore"), ntile(5).over(Window.partitionBy().orderBy(rfm_overview_df.Frequency)).alias("FrequencyScore"), ntile(5).over(Window.partitionBy().orderBy(rfm_overview_df.Monetary)).alias("MonetaryScore"))
             .orderBy("CustomerID")
            )

# Display the resulting dataframe
display(rfm_table)

CustomerID,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore
12346.0,4074,12,77556.46,4,5,5
12347.0,3751,8,5633.3199999999915,1,4,5
12348.0,3824,5,2019.4,3,4,4
12349.0,3767,4,4428.690000000002,1,3,5
12350.0,4059,1,334.40000000000003,4,1,2
12351.0,4124,1,300.93,4,1,2
12352.0,3785,10,2849.8400000000015,2,5,4
12353.0,3953,2,406.75999999999993,4,2,2
12354.0,3981,1,1079.4,4,1,3
12355.0,3963,2,947.61,4,2,3


In [0]:
# Import chain
from itertools import chain

# Reverse 1-5 scale to 5-1 scale in RecencyScore
recency_dict = { 1:5, 2:4, 3:3, 4:2, 5:1 }
mapping = create_map([lit(x) for x in chain(*recency_dict.items())])
rfm_table_scores = rfm_table.select("CustomerID", "Recency", "Frequency", "Monetary", mapping[rfm_table["RecencyScore"]].alias("RecencyScore"), "FrequencyScore", "MonetaryScore")

# Add RFMScore column
rfm_table_scores = rfm_table_scores.withColumn("RFMScore", regexp_replace(concat(rfm_table_scores.RecencyScore.cast("String"), rfm_table_scores.FrequencyScore.cast("String"), rfm_table_scores.MonetaryScore.cast("String")), ".0", "").cast("Integer"))

# Display the resulting dataframe
display(rfm_table_scores)

CustomerID,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFMScore
12346.0,4074,12,77556.46,2,5,5,255
12347.0,3751,8,5633.3199999999915,5,4,5,545
12348.0,3824,5,2019.4,3,4,4,344
12349.0,3767,4,4428.690000000002,5,3,5,535
12350.0,4059,1,334.40000000000003,2,1,2,212
12351.0,4124,1,300.93,2,1,2,212
12352.0,3785,10,2849.8400000000015,4,5,4,454
12353.0,3953,2,406.75999999999993,2,2,2,222
12354.0,3981,1,1079.4,2,1,3,213
12355.0,3963,2,947.61,2,2,3,223


In [0]:
# Create a map object to label segments by RFM score
seg_map = {
    '11': 'Hibernating',
    '12': 'Hibernating',
    '21': 'Hibernating',
    '22': 'Hibernating',
    '13': 'At Risk',
    '14': 'At Risk',
    '23': 'At Risk',
    '24': 'At Risk',
    '15': 'Can\'t Lose',
    '25': 'Can\'t Lose',
    '31': 'About to Sleep',
    '32': 'About to Sleep',
    '33': 'Need Attention',
    '34': 'Loyal Customer',
    '44': 'Loyal Customer',
    '35': 'Loyal Customer',
    '45': 'Loyal Customer',
    '41': 'Promising',
    '51': 'New Customers',
    '42': 'Potential Loyalists',
    '43': 'Potential Loyalists',
    '52': 'Potential Loyalists',
    '53': 'Potential Loyalists',
    '54': 'Champions',
    '55': 'Champions'
}

# Create a mapping expression
mapping_expr = create_map([lit(x) for x in chain(*seg_map.items())])

In [0]:
# Add a "Segment" column
rfm_table_segments = rfm_table_scores.withColumn("Segment", regexp_replace(concat(rfm_table_scores.RecencyScore.cast("String"), rfm_table_scores.FrequencyScore.cast("String")), ".0", "").cast("String"))
rfm_table_segments = rfm_table_segments.withColumn("Segment", mapping_expr.getItem(col("Segment")))

# Display the resulting dataframe
display(rfm_table_segments)

CustomerID,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFMScore,Segment
12346.0,4074,12,77556.46,2,5,5,255,Can't Lose
12347.0,3751,8,5633.3199999999915,5,4,5,545,Champions
12348.0,3824,5,2019.4,3,4,4,344,Loyal Customer
12349.0,3767,4,4428.690000000002,5,3,5,535,Potential Loyalists
12350.0,4059,1,334.40000000000003,2,1,2,212,Hibernating
12351.0,4124,1,300.93,2,1,2,212,Hibernating
12352.0,3785,10,2849.8400000000015,4,5,4,454,Loyal Customer
12353.0,3953,2,406.75999999999993,2,2,2,222,Hibernating
12354.0,3981,1,1079.4,2,1,3,213,Hibernating
12355.0,3963,2,947.61,2,2,3,223,Hibernating


In [0]:
# Create a dataframe containing mean and count of each segment
rfm_table_segments_agg = (rfm_table_segments
                          .select("Segment", "Recency", "Frequency", "Monetary")
                          .groupby("Segment").agg(mean("Recency").alias("MeanRecency"),
                                                  count("Recency").alias("CountRecency"),
                                                  mean("Frequency").alias("MeanFrequency"),
                                                  count("Frequency").alias("CountFrequency"),
                                                  mean("Monetary").alias("MeanMonetary"),
                                                  count("Monetary").alias("CountMonetary")
                                                 )
                          .orderBy("Segment")
                         )

# Display the resulting dataframe
display(rfm_table_segments_agg)

Segment,MeanRecency,CountRecency,MeanFrequency,CountFrequency,MeanMonetary,CountMonetary
About to Sleep,3854.9766233766236,385,1.3558441558441559,385,531.6770207792208,385
At Risk,4121.159362549801,753,3.909694555112882,753,1382.2359760956174,753
Can't Lose,4079.8028169014083,71,15.929577464788732,71,8355.677633802808,71
Champions,3756.8576642335765,822,19.482968369829685,822,11036.223132603403,822
Hibernating,4207.329619921363,1526,1.254259501965924,1526,438.52393315858416,1526
Loyal Customer,3814.865531914894,1175,9.862127659574469,1175,4205.459817021268,1175
Need Attention,3861.674157303371,267,3.157303370786517,267,1283.9465917602997,267
New Customers,3759.3035714285716,56,1.0,56,356.2578571428571,56
Potential Loyalists,3774.297335203366,713,2.5890603085554,713,1155.427280504909,713
Promising,3787.2818181818184,110,1.0,110,324.49699999999996,110


In [0]:
# Create a dataframe containing segment and mean recency
rfm_table_segments_recency = (rfm_table_segments_agg
                              .select("Segment", "MeanRecency")
                              .orderBy("MeanRecency", ascending=[1])
                             )

# Display the resulting dataframe
display(rfm_table_segments_recency)

Segment,MeanRecency
Champions,3756.8576642335765
New Customers,3759.3035714285716
Potential Loyalists,3774.297335203366
Promising,3787.2818181818184
Loyal Customer,3814.865531914894
About to Sleep,3854.9766233766236
Need Attention,3861.674157303371
Can't Lose,4079.8028169014083
At Risk,4121.159362549801
Hibernating,4207.329619921363


In [0]:
# Create a dataframe containing segment and mean frequency
rfm_table_segments_frequency = (rfm_table_segments_agg
                              .select("Segment", "MeanFrequency")
                              .orderBy("MeanFrequency", ascending=[0])
                             )

# Display the resulting dataframe
display(rfm_table_segments_frequency)

Segment,MeanFrequency
Champions,19.482968369829685
Can't Lose,15.929577464788732
Loyal Customer,9.862127659574469
At Risk,3.909694555112882
Need Attention,3.157303370786517
Potential Loyalists,2.5890603085554
About to Sleep,1.3558441558441559
Hibernating,1.254259501965924
New Customers,1.0
Promising,1.0


In [0]:
# Create a dataframe containing segment and mean monetary
rfm_table_segments_monetary = (rfm_table_segments_agg
                              .select("Segment", "MeanMonetary")
                              .orderBy("MeanMonetary", ascending=[0])
                             )

# Display the resulting dataframe
display(rfm_table_segments_monetary)

Segment,MeanMonetary
Champions,11036.223132603403
Can't Lose,8355.677633802808
Loyal Customer,4205.459817021268
At Risk,1382.2359760956174
Need Attention,1283.9465917602997
Potential Loyalists,1155.427280504909
About to Sleep,531.6770207792208
Hibernating,438.52393315858416
New Customers,356.2578571428571
Promising,324.49699999999996


In [0]:
# Find the total customers
total_customers = rfm_table_segments.select("CustomerID").distinct().count()
print(total_customers)

5878

In [0]:
# Create a UDF to find the percentage of total
from pyspark.sql.types import StringType
percentage_total = udf(lambda x: ((100 * x) / total_customers), StringType())

# Create a dataframe containing segments and the number of customers in each segment
segment_counts = (rfm_table_segments
                  .select("Segment", "CustomerID")
                  .groupBy("Segment").agg(count("CustomerID"))
                  .withColumnRenamed("count(CustomerID)", "CustomerIDCount")
                 )

# Add a percentage total column
segment_counts = (segment_counts
                  .withColumn("PercentageTotal", percentage_total(segment_counts.CustomerIDCount))
                  .orderBy("CustomerIDCount", ascending=[0])
                 )

# Display the resulting dataframe
display(segment_counts)

Segment,CustomerIDCount,PercentageTotal
Hibernating,1526,25.961211296359306
Loyal Customer,1175,19.989792446410345
Champions,822,13.984348417829194
At Risk,753,12.810479755018713
Potential Loyalists,713,12.129976182374957
About to Sleep,385,6.549846886696155
Need Attention,267,4.542361347397073
Promising,110,1.87138482477033
Can't Lose,71,1.2078938414426676
New Customers,56,0.9527050017012589
